# Laboratorium 01 — Wprowadzenie do analizy obrazów medycznych (DICOM i NIfTI)

W tym laboratorium rozpoczniemy pracę z medycznymi danymi obrazowymi. Poznamy podstawowe formaty (DICOM, NIfTI), sposoby wczytywania i eksploracji wolumenów 3D, przegląd najważniejszych metadanych oraz wykonamy proste przekształcenia i wizualizacje.

## Cele nauczania
- Zrozumienie różnic między formatami DICOM i NIfTI oraz kiedy ich używać.
- Wczytywanie obrazów z użyciem bibliotek: pydicom, SimpleITK, nibabel, nilearn.
- Odczyt i interpretacja metadanych.
- Podstawowe operacje na danych 3D: wybór przekrojów, progowanie, obliczanie gradientów (Sobel), przepróbkowanie (interpolacja danych).
- Przegląd i zapis danych do plików NIfTI.

## Zakres i przepływ pracy
1. DICOM: wczytanie pojedynczych plików, przegląd nagłówka i pikseli (pydicom, SimpleITK)
2. NIfTI: wczytanie wolumenu, macierz affine, orientacje, podstawowe przekroje (nibabel)
3. Wizualizacja 2D/3D: przekroje osiowe/koronalne/strzałkowe, proste układy figur (matplotlib, opcjonalnie nilearn)
4. Proste przekształcenia: progowanie, filtr Sobela 3D, filtracja medianowa
5. Resamplowanie i zapis: izotropizacja voxeli, zapis przetworzonych danych do NIfTI

## Dane do ćwiczeń
Dane powinny pobrać się automatycznie w ramach skryptu. Znajdują się one w folderze data lub /content/data (w przypadku Google Colab).

## Wymagane narzędzia i biblioteki
- Python 3.9+ oraz:
  - numpy, matplotlib
  - pydicom, SimpleITK
  - nibabel, nilearn
- Biblioteki należy zainstalować na początku skryptu jeśli pracujesz w Google Colab. Jeśli pracujesz lokalnie, to warto stworzyć środowisko conda lub venv.

## Typowe pułapki i wskazówki
- Orientacje i układy współrzędnych: DICOM zwykle używa LPS, wiele narzędzi neuro (nibabel/nilearn) preferuje RAS. Nie mieszaj konwencji bez kontroli affine.
- Jednostki i skale intensywności: CT (HU - Hounsfield Units) vs MR (wartości względne). Normalizuj lub standaryzuj dane przed progowaniem/filtracją.
- Spójny spacing: przed analizą porównawczą rozważ resamplowanie do voxeli izotropowych (np. 1×1×1 mm).
- Pamięć i wydajność: pracujemy na małych przykładach, ale na większych danych stosuje się lazy loading (dane nie są wczytywane od razu, tylko po wywołaniu), ogranicza rozmiar figur i liczbę kopii tablic.

Powodzenia! W kolejnych sekcjach przejdziemy od wczytania plików po zapis przetworzonych wolumenów i krótką analizę.

## 0) Instalacja bibliotek (jeśli potrzebne)

In [ ]:
# Uruchom, jeśli pojawi się błąd 'ModuleNotFoundError'
!pip -q install scipy nibabel nilearn SimpleITK pydicom matplotlib numpy


## 1) Automatyczne pobranie przykładowych danych

Aby nie trzeba było samodzielnie uploadować danych, wczytamy:
- **DICOM**: przykładowe pliki z pakietu `pydicom` (CT, MR),
- **NIfTI**: mały wolumen MNI152 (szablon MRI mózgu) pobrany z `nilearn`.


In [ ]:
import os, shutil, urllib.request
import numpy as np
import matplotlib.pyplot as plt

import nibabel as nib
import SimpleITK as sitk
import pydicom
from pydicom.data import get_testdata_files

from scipy import ndimage as ndi
import cv2

BASE = "/content/data" if os.path.isdir("/content") else "./data"
DATA_DIR = os.path.join(BASE, "lab01_data")
os.makedirs(DATA_DIR, exist_ok=True)

# --- DICOM ---
dicom_dir = os.path.join(DATA_DIR, "dicom")
os.makedirs(dicom_dir, exist_ok=True)

dicom_examples = ["CT_small.dcm", "MR_small.dcm", "SC_rgb.dcm"]
for name in dicom_examples:
    src = get_testdata_files(name)[0]
    dst = os.path.join(dicom_dir, name)
    shutil.copy(src, dst)
print("Skopiowano pliki DICOM:", os.listdir(dicom_dir))

# --- NIfTI ---
nifti_dir = os.path.join(DATA_DIR, "nifti")
os.makedirs(nifti_dir, exist_ok=True)

nifti_path = os.path.join(nifti_dir, "mni152.nii.gz")
try:
    from nilearn.datasets import load_mni152_template
    img = load_mni152_template(resolution=2)
    nib.save(img, nifti_path)
    print("Pobrano MNI152 przez nilearn:", nifti_path)
except Exception as e:
    print("Błąd nilearn:", e)
    url = "https://nipy.org/nibabel/_static/nifti_examples/nifti1.nii.gz"
    urllib.request.urlretrieve(url, nifti_path)
    print("Pobrano fallback NIfTI:", nifti_path)

print("Gotowe dane w:", DATA_DIR)



## Zadanie 1 – Wczytanie obrazu DICOM i metadane

1. Wykonaj ponizsze operacje dla każdego z pobranych plików testowych DICOM (nazwy w `dicom_examples`).
2. Metadane mogą być wczytane zarówno za pomocą biblioteki `pydicom` jak i `SimpleITK`.
3. Najpierw wczytaj plik za pomocą funkcji `pydicom.dcmread`.
4. Następnie wczytaj i wyświetl za pomocą funkcji `getattr` metadane takie jak: `"Modality", "Rows", "Columns", "PixelSpacing", "SliceThickness", "Manufacturer"`.
Przykład wczytania przykładowej wartości: `print("Photometric Interpretation:", getattr(img_dcm_pydicom, "PhotometricInterpretation", "brak"))`.
5. Wczytaj ten sam plik za pomocą funkcji `sitk.ReadImage`.
6. Wypisz wszystkie dostępne metadane dostępne w plikach. W tym celu wykorzystaj funkcji `img.GetMetaDataKeys()` oraz `img_dcm.GetMetaData(key)`.
7. Przeanalizuj dostępne metadane.
8. Wczytaj dane obrazowe za pomocą funkcji `sitk.GetArrayFromImage`, a następnie wyświetl dane za pomocą `matplotlib`.

In [ ]:
# --- Zadanie 1
dcm_paths = ["/content/data/lab01_data/dicom/CT_small.dcm", "/content/data/lab01_data/dicom/MR_small.dcm", "/content/data/lab01_data/dicom/SC_rgb.dcm"]

for file in dcm_paths:
    print("="*40)
    print(f"Plik: {file}")

    img_dcm_pydicom = pydicom.dcmread(file)

    for tag in ["Modality", "Rows", "Columns", "PixelSpacing", "SliceThickness", "Manufacturer"]:
        print(f"{tag}:", getattr(img_dcm_pydicom, tag, "brak"))

    img_sitk = sitk.ReadImage(file)

    print("\n--- Dostępne metadane (SimpleITK) ---")
    for key in img_sitk.GetMetaDataKeys():
        value = img_sitk.GetMetaData(key)
        print(f"{key}: {value}")

    img_array = sitk.GetArrayFromImage(img_sitk)

    if img_array.ndim > 2:
        slice_to_show = img_array[0, :, :]
    else:
        slice_to_show = img_array

    # --- 6. Wyświetlenie obrazu
    plt.imshow(slice_to_show, cmap="gray")
    plt.title(f"Podgląd obrazu DICOM: {file}")
    plt.axis("off")
    plt.show()


## Zadanie 2 – Wczytanie wolumenu NIfTI i przekroje

1. Wczytaj plik `mni152.nii.gz`. Ścieżka znajduje się w `nifti_path`. Wykorzystaj funkcję `nib.load`.
2. Wczytaj dane za pomocą funkcji `file.get_fdata()`.
3. Sprawdź kształt tablicy (`data.shape`), macierz affine (`file.affine`), spacing (`file.header.get_zooms()`) oraz kierunek osi `nib.aff2axcodes(nii.affine)`.
4. Wyświetl trzy środkowe przekroje: axial (poprzeczny - "od góry"), coronal (czołowy - "od przodu"), sagittal (strzałkowy - "od boku").


In [ ]:

mni152_nii_gz = nib.load(nifti_path)

fdata = mni152_nii_gz.get_fdata()

print(f"Data shape: {fdata.shape}")
print(f"Affine matrix:\n{mni152_nii_gz.affine}")
print(f"Spacing: {mni152_nii_gz.header.get_zooms()}")
print(f"Axis vector: {nib.aff2axcodes(mni152_nii_gz.affine)}")

x_mid = fdata.shape[0] // 2  # sagittal
y_mid = fdata.shape[1] // 2  # coronal
z_mid = fdata.shape[2] // 2  # axial

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Axial (od góry)
axes[0].imshow(fdata[:, :, z_mid].T, cmap="gray", origin="lower")
axes[0].set_title("Axial (od góry)")
axes[0].axis("off")

# Coronal (od przodu)
axes[1].imshow(fdata[:, y_mid, :].T, cmap="gray", origin="lower")
axes[1].set_title("Coronal (od przodu)")
axes[1].axis("off")

# Sagittal (od boku)
axes[2].imshow(fdata[x_mid, :, :].T, cmap="gray", origin="lower")
axes[2].set_title("Sagittal (od boku)")
axes[2].axis("off")

plt.show()


## Zadanie 3 – Histogram i podstawowe operacje

1. Wyświetl histogram intensywności wolumenu (NIfTI) za pomocą funkcji `plt.hist`. W celu poprawy widoczności można ustawić `range=(0.01, 1)`. Wcześniej spłaszcz dane za pomocą metody `.ravel()`.
2. Wyświetl 3 oryginalne przekroje (jak w poprzednim zadaniu).
3. Oblicz gradient Sobela względem wszystkich trzech osi. Wykorzystaj funkcję `ndi.sobel`.
4. Za pomocą normy L2 połącz wyniki z wszystkich osi.
5. Wyświetl te same przekroje jak wcześniej.
6. Wykonaj progowanie dla 90-tego precentyla danych. Do wyznaczenia tej wartości wykorzystaj funkcję `np.percentile`.
7. Wykonaj progowania, a następnie zrzutuj wynik do typu `np.uint8`.
8. Wyświetl te same przekroje jak wcześniej.
9. Wykonaj filtrację medianową dla tych samych danych. Wykorzystaj funkcję `ndi.median_filter`.
10. Wyświetl te same przekroje jak wcześniej.


In [ ]:
# --- Zadanie 3
plt.hist(fdata.flatten(), range=(0.01, 1))

x_mid = fdata.shape[0] // 2  # sagittal
y_mid = fdata.shape[1] // 2  # coronal
z_mid = fdata.shape[2] // 2  # axial

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axial = fdata[:, :, z_mid].T
coronal = fdata[:, y_mid, :].T
sagittal = fdata[x_mid, :, :].T

# Axial (od góry)
axes[0].imshow(ndi.sobel(fdata[:, :, z_mid].T), cmap="gray", origin="lower")
axes[0].set_title("Axial (od góry)")
axes[0].axis("off")

# Coronal (od przodu)
axes[1].imshow(ndi.sobel(fdata[:, y_mid, :].T), cmap="gray", origin="lower")
axes[1].set_title("Coronal (od przodu)")
axes[1].axis("off")

# Sagittal (od boku)
axes[2].imshow(ndi.sobel(fdata[x_mid, :, :].T), cmap="gray", origin="lower")
axes[2].set_title("Sagittal (od boku)")
axes[2].axis("off")

plt.show()

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Axial (od góry)
axes[0].imshow(np.linalg.norm(fdata.T, axis=0), cmap="gray", origin="lower")
axes[0].set_title("Axial (od góry)")
axes[0].axis("off")

# Coronal (od przodu)
axes[1].imshow(np.linalg.norm(fdata.T, axis=1), cmap="gray", origin="lower")
axes[1].set_title("Coronal (od przodu)")
axes[1].axis("off")

# Sagittal (od boku)
axes[2].imshow(np.linalg.norm(fdata.T, axis=2), cmap="gray", origin="lower")
axes[2].set_title("Sagittal (od boku)")
axes[2].axis("off")

plt.show()


fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# --- obliczenie progów (90 percentyl)
z_perc = np.percentile(axial, 90)
y_perc = np.percentile(coronal, 90)
x_perc = np.percentile(sagittal, 90)

# --- thresholdowanie (cv2 zwraca: (ret, obraz))
_, axial_thresh    = cv2.threshold(axial.astype(np.float32),    z_perc, 1, cv2.THRESH_BINARY)
_, coronal_thresh  = cv2.threshold(coronal.astype(np.float32),  y_perc, 1, cv2.THRESH_BINARY)
_, sagittal_thresh = cv2.threshold(sagittal.astype(np.float32), x_perc, 1, cv2.THRESH_BINARY)

# --- wyświetlanie
axes[0].imshow(axial_thresh.astype(np.uint8), cmap="gray", origin="lower")
axes[0].set_title("Axial (threshold 90%)")
axes[0].axis("off")

axes[1].imshow(coronal_thresh.astype(np.uint8), cmap="gray", origin="lower")
axes[1].set_title("Coronal (threshold 90%)")
axes[1].axis("off")

axes[2].imshow(sagittal_thresh.astype(np.uint8), cmap="gray", origin="lower")
axes[2].set_title("Sagittal (threshold 90%)")
axes[2].axis("off")

plt.show()

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Axial (od góry)
axes[0].imshow(ndi.median_filter(fdata[:, :, z_mid].T, size=(5, 5)), cmap="gray", origin="lower")
axes[0].set_title("Axial (od góry, median filter)")
axes[0].axis("off")

# Coronal (od przodu)
axes[1].imshow(ndi.median_filter(fdata[:, y_mid, :].T, size=(5, 5)), cmap="gray", origin="lower")
axes[1].set_title("Coronal (od przodu, median filter)")
axes[1].axis("off")

# Sagittal (od boku)
axes[2].imshow(ndi.median_filter(fdata[x_mid, :, :].T, size=(5, 5)), cmap="gray", origin="lower")
axes[2].set_title("Sagittal (od boku, median filter)")
axes[2].axis("off")

plt.show()



## Zadanie 4 – Resampling wolumenu do izotropowego voxela

1. Wczytaj dane NIfTI za pomocą `SimpleITK`. Wykorzystaj funkcję `sitk.ReadImage`.
2. Wyświetl rozmiar i spacing wczytanych danych. Wykorzystaj metody `.GetSize()` i `.GetSpacing()`.
3. Stwórz instancję klasy `sitk.ResampleImageFilter()`.
4. Wybierz metodę interpolacji za pomocą metody `.SetInterpolator`.
5. Wybierz wyjściowy spacing za pomocą metody `.SetOutputSpacing`. Chcemy uzyskać spacing 1mm dla każdej osi.
6. Ustaw orientację i położenie początka ukłądu współrzędnych jak w oryginalnych danych. Wykorzystaj metody `.SetOutputDirection`, `.GetDirection`, `.SetOutputOrigin` i `.GetOrigin`.
7. Oblicz rozmiar wyjściowych danych. Rozmiar dla analizowanego wymiaru powinien wynosić $Sz_o \cdot Sp_o / Sp_n$, gdzie $Sz_o$ jest originalnym rozmiarem, $Sp_o$ jest oryginalnym spacingiem, a $Sp_n$ jest docelowym spacingiem.
8. Ustaw docelowy rozmiar za pomocą metody `.SetSize`.
9. Wykonaj przeprobkowanie danych za pomocą metody `.Execute`.
10. Wyświetl rozmiar oraz spacing wyjściowych danych.

In [ ]:
# 1. Wczytaj dane NIfTI
nifTI_image = sitk.ReadImage(nifti_path)

# 2. Wyświetl rozmiar i spacing
print("Oryginalny rozmiar:", nifTI_image.GetSize())
print("Oryginalny spacing:", nifTI_image.GetSpacing())

# 3. Stwórz filtr resamplingu
res_img_fltr = sitk.ResampleImageFilter()

# 4. Ustaw interpolację (np. B-spline)
res_img_fltr.SetInterpolator(sitk.sitkBSpline)

# 5. Docelowy spacing (1mm x 1mm x 1mm)
new_spacing = (1.0, 1.0, 1.0)
res_img_fltr.SetOutputSpacing(new_spacing)

# 6. Zachowaj orientację i origin
res_img_fltr.SetOutputDirection(nifTI_image.GetDirection())
res_img_fltr.SetOutputOrigin(nifTI_image.GetOrigin())

# 7. Oblicz nowy rozmiar
old_size = np.array(nifTI_image.GetSize(), dtype=np.int64)
old_spacing = np.array(nifTI_image.GetSpacing())
new_spacing_np = np.array(new_spacing)

new_size = np.round(old_size * (old_spacing / new_spacing_np)).astype(int)
new_size = [int(sz) for sz in new_size]  # konwersja do listy int
res_img_fltr.SetSize(new_size)

# 9. Wykonaj resampling
resampled_img = res_img_fltr.Execute(nifTI_image)

# 10. Wyświetl wynikowe parametry
print("Nowy rozmiar:", resampled_img.GetSize())
print("Nowy spacing:", resampled_img.GetSpacing())


## Zadanie 5 – Zapis przetworzonego wolumenu w formacie NIfTI (LPS -> RAS)

1. Utwórz ścieżkę gdzie dane zostaną zapisane.
2. Wyciągnij dane `NumPy` z obrazu `SimpleITK`.
3. Dopasuj kolejność wymiarów do biblioteki `nibabel` - (z, y, x) -> (x, y, z). Wykorzystaj funkcję `np.transpose`.
4. Pobierz spacing, direction i origin dla zapisywanych danych. Zamień je na `np.array` typu `float`. Zmień kształt direction na `(3, 3)` metodą `.reshape`.
5. Wyświetl pobrane wartości.
6. Najpierw stwórz macierz affine dla formatu LPS (stosowany w `SimpleITK` i DICOM). Zacznij od stworzenia macierzy jednostkowej o rozmiarze $4 \times 4$. Następnie w trzy pierwsze kolumny i wiersze wpisz kolumny direction przeskalowane przez spacing (wystarczy operacja mnożenia element przez element - `*`). Do trzech pierwszych wierszy ostatniej kolumny wpisz origin.
7. Skonwertuj affine w formacie LPS do formatu RAS. W tym celu wystarczy odwrócić kierunki osi x i y. Można to zrobić wykonująć mnożenie macierzowe formatu lps przez macierz diagonalną o wartościach `[-1, -1, 1, 1]`. macierz diagonalną można stworzyć za pomocą `np.diag`.
8. Wyświetl obliczone affine. Porównaj go z affine oryginalnych danych, wczytanych na początku zadania 2.
9. Stwórz obraz NIfTI za pomocą `nib.Nifti1Image`. Pierwszym argumentem są dane jako tablica `NumPy`, a drugim affine w formacie RAS.
10. Wykonaj dodatkowo `nifti.set_sform(affine_ras, code=1)` i `nifti.set_qform(affine_ras, code=1)`.
11. Zapisz dane za pomocą `nib.save`.

In [ ]:
# --- Zadanie 5

output_path = "/content/data/lab01_data/output"
os.makedirs(output_path, exist_ok=True)

img_array = sitk.GetArrayFromImage(nifTI_image)  # (z, y, x)

img_array_xyz = np.transpose(img_array, (2, 1, 0))

spacing = np.array(nifTI_image.GetSpacing(), dtype=float)
direction = np.array(nifTI_image.GetDirection(), dtype=float).reshape(3, 3)
origin = np.array(nifTI_image.GetOrigin(), dtype=float)

print("Spacing:", spacing)
print("Direction:\n", direction)
print("Origin:", origin)

affine_lps = np.eye(4)
affine_lps[:3, :3] = direction * spacing
affine_lps[:3, 3] = origin

lps_to_ras = np.diag([-1, -1, 1, 1])
affine_ras = lps_to_ras @ affine_lps

print("Affine LPS:\n", affine_lps)
print("Affine RAS:\n", affine_ras)
print("Affine z nibabel (oryginalny, zadanie 2):\n", mni152_nii_gz.affine)

nifti_out = nib.Nifti1Image(img_array_xyz, affine_ras)

nifti_out.set_sform(affine_ras, code=1)
nifti_out.set_qform(affine_ras, code=1)

save_path = os.path.join(output_path, "resampled_LPS_to_RAS.nii.gz")
nib.save(nifti_out, save_path)

print(f"NIfTI zapisany do: {save_path}")

mni152_nii_gz = nib.load(save_path)

fdata = mni152_nii_gz.get_fdata()

print(f"Data shape: {fdata.shape}")
print(f"Affine matrix:\n{mni152_nii_gz.affine}")
print(f"Spacing: {mni152_nii_gz.header.get_zooms()}")
print(f"Axis vector: {nib.aff2axcodes(mni152_nii_gz.affine)}")

x_mid = fdata.shape[0] // 2  # sagittal
y_mid = fdata.shape[1] // 2  # coronal
z_mid = fdata.shape[2] // 2  # axial

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Axial (od góry)
axes[0].imshow(fdata[:, :, z_mid].T, cmap="gray", origin="lower")
axes[0].set_title("Axial (od góry)")
axes[0].axis("off")

# Coronal (od przodu)
axes[1].imshow(fdata[:, y_mid, :].T, cmap="gray", origin="lower")
axes[1].set_title("Coronal (od przodu)")
axes[1].axis("off")

# Sagittal (od boku)
axes[2].imshow(fdata[x_mid, :, :].T, cmap="gray", origin="lower")
axes[2].set_title("Sagittal (od boku)")
axes[2].axis("off")

plt.show()


---
## Podsumowanie (do krótkiego uzupełnienia)

- Jakie są główne różnice między DICOM a NIfTI?
DICOM to format kliniczny, bogaty w metadane o pacjencie i aparacie, zwykle zawierający wiele plików dla jednego badania.
NIfTI to format badawczy, uproszczony i jednoplikowy, skupiony na danych przestrzennych i analizie neuroobrazowej.

- Jakie metadane są ważne przy pracy z obrazami medycznymi?
Kluczowe są informacje o wymiarach, rozdzielczości, orientacji i parametrach akwizycji (np. voxel size).

- Jak interpretować trzy podstawowe przekroje wolumenu (axial, coronal, sagittal)?

Axial zgodnie z płaszczyną dysków międzykręgowych, coronal poprzeczny, sagittal zgodnie z płaszczyzną przechodzącą kolinearnie przez mostek i kręgosłup.